# Тематическая классификация длинных текстов - TFIDF и LogReg

In [1]:
# # Если Вы запускаете ноутбук на colab или kaggle,
# # выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

In [2]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

# import dlnlputils
# from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
#     vectorize_texts, SparseFeaturesDataset
# from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

# init_random_seed()

## Предобработка текстов и подготовка признаков

In [3]:
train_source = fetch_20newsgroups(subset='train')
test_source = fetch_20newsgroups(subset='test')

print('Количество обучающих текстов', len(train_source['data']))
print('Количество тестовых текстов', len(test_source['data']))
print()
print(train_source['data'][0].strip())

print()
print('Метка', train_source['target'][0])

Количество обучающих текстов 11314
Количество тестовых текстов 7532

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----

Метка 7


In [4]:
train_source.__dir__()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

### Подготовка признаков

In [5]:
# train_tokenized = tokenize_corpus(train_source['data'])
# test_tokenized = tokenize_corpus(test_source['data'])

# print(' '.join(train_tokenized[0]))

In [6]:
# # максимальная частотность слова в документе
# MAX_DF = 0.8
# MIN_COUNT = 5
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
# UNIQUE_WORDS_N = len(vocabulary)
# print('Количество уникальных токенов', UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])

In [7]:
# plt.hist(word_doc_freq, bins=50)
# plt.title('Распределение относительных частот слов')
# plt.yscale('log');

In [8]:
# VECTORIZATION_MODE = 'tfidf'
# train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
# test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

# print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
# print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
# print()
# print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
# print()
# print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
# print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [9]:
# plt.hist(train_vectors.data, bins=20)
# plt.title('Распределение весов признаков')
# plt.yscale('log');

### Распределение классов

In [10]:
# UNIQUE_LABELS_N = len(set(train_source['target']))
# print('Количество уникальных меток', UNIQUE_LABELS_N)

In [11]:
# plt.hist(train_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в обучающей выборке');

In [12]:
# plt.hist(test_source['target'], bins=np.arange(0, 21))
# plt.title('Распределение меток в тестовой выборке');

### PyTorch Dataset

In [13]:
# from torch.utils.data import Dataset

# class SparseFeaturesDataset(Dataset):
#     def __init__(self, features, targets):
#         self.features = features
#         self.targets = targets

#     def __len__(self):
#         return self.features.shape[0]

#     def __getitem__(self, idx):
#         # print(self.features[idx].toarray().shape)
#         cur_features = torch.from_numpy(self.features[idx].toarray().squeeze()).float()
#         cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
#         return cur_features, cur_label

In [14]:
# train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
# test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

## Обучение модели на PyTorch

In [15]:
# model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

# scheduler = lambda optim: \
#     torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

# best_val_loss, best_model = train_eval_loop(model=model,
#                                             train_dataset=train_dataset,
#                                             val_dataset=test_dataset,
#                                             criterion=F.cross_entropy,
#                                             lr=1e-1,
#                                             epoch_n=200,
#                                             batch_size=32,
#                                             l2_reg_alpha=0,
#                                             lr_scheduler_ctor=scheduler)

## Оценка качества

In [16]:
# train_pred = predict_with_model(best_model, train_dataset)

# train_loss = F.cross_entropy(torch.from_numpy(train_pred),
#                              torch.from_numpy(train_source['target']).long())

# print('Среднее значение функции потерь на обучении', float(train_loss))
# print('Доля верных ответов', accuracy_score(train_source['target'], train_pred.argmax(-1)))
# print()



# test_pred = predict_with_model(best_model, test_dataset)

# test_loss = F.cross_entropy(torch.from_numpy(test_pred),
#                             torch.from_numpy(test_source['target']).long())

# print('Среднее значение функции потерь на валидации', float(test_loss))
# print('Доля верных ответов', accuracy_score(test_source['target'], test_pred.argmax(-1)))

## Оценка качества - precision и recall

In [17]:
# print('Precision', precision_score(train_source['target'], train_pred.argmax(-1), average="weighted"))
# print('Recall', recall_score(train_source['target'], train_pred.argmax(-1), average="weighted"))
# print('F1score', f1_score(train_source['target'], train_pred.argmax(-1), average="weighted"))

In [18]:
# print('Precision', precision_score(test_source['target'], test_pred.argmax(-1), average="weighted"))
# print('Recall', recall_score(test_source['target'], test_pred.argmax(-1), average="weighted"))
# print('F1score', f1_score(test_source['target'], test_pred.argmax(-1), average="weighted"))

# Альтернативная реализация на scikit-learn

In [19]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression

# sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
#                                                       max_df=MAX_DF,
#                                                       min_df=MIN_COUNT)),
#                              ('cls', LogisticRegression())))
# sklearn_pipeline.fit(train_source['data'], train_source['target']);

## Оценка качества

In [20]:
# sklearn_train_pred = sklearn_pipeline.predict_proba(train_source['data'])
# sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
#                                                  torch.from_numpy(train_source['target']))
# print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
# print('Доля верных ответов', accuracy_score(train_source['target'], sklearn_train_pred.argmax(-1)))
# print()

# sklearn_test_pred = sklearn_pipeline.predict_proba(test_source['data'])
# sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
#                                                 torch.from_numpy(test_source['target']))
# print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
# print('Доля верных ответов', accuracy_score(test_source['target'], sklearn_test_pred.argmax(-1)))

## Оценка качества - precision и recall

In [21]:
# print('Precision', precision_score(train_source['target'], sklearn_train_pred.argmax(-1), average="weighted"))
# print('Recall', recall_score(train_source['target'], sklearn_train_pred.argmax(-1), average="weighted"))
# print('F1score', f1_score(train_source['target'], sklearn_train_pred.argmax(-1), average="weighted"))

In [22]:
# print('Precision', precision_score(test_source['target'], sklearn_test_pred.argmax(-1), average="weighted"))
# print('Recall', recall_score(test_source['target'], sklearn_test_pred.argmax(-1), average="weighted"))
# print('F1score', f1_score(test_source['target'], sklearn_test_pred.argmax(-1), average="weighted"))

## TODO:
Поэкспериментировать с собственноручно написанной моделью и попытаться уменьшить эффект переобучения
- Обработка данных:
-- N-грамы, лемматизацию, токенизацию
-- HashigVectorizer или StandardScaler
-- Векторизация на основе PMI (Pointwise mutual information)

Пробуем TfIDF векторизацию для 1-3-грам с токенизацией c лемматизацией(не показала себя) и скейлингом

Давайте хотя бы взглянем а какие классы текстов у нас есть

In [23]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

import nltk

In [24]:
type(train_source['data'])
train_corpus = train_source['data']
test_corpus = test_source['data']

In [25]:
class VECTORIZATION_CONSTS:
  def __init__(self):
    self.MAX_DF=0.8
    self.MIN_DF=5
    self.NGRAM_RANGE=(1,2)

VECT_CONSTS = VECTORIZATION_CONSTS()

In [26]:
VECT_CONSTS.MAX_DF

0.8

In [27]:
# import nltk
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import issparse

class ColumnNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if issparse(X):
            # Normalize each column of the sparse matrix
            X = X.tocsc()
            col_norms = np.sqrt(X.multiply(X).sum(axis=0))
            col_norms[col_norms == 0] = 1  # Avoid division by zero
            X /= col_norms
            return X.tocsr()
        else:
            # If X is not sparse, you can use StandardScaler
            scaler = StandardScaler(with_mean=False)
            return scaler.fit_transform(X)

In [29]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# matrix = np.array([[1, 2, 3],
#                    [4, 5, 6],
#                    [7, 8, 9]])

# # Define the pipeline
# pipeline = Pipeline([
#     ('normalizer', ColumnNormalizer()),
# ])

# # Apply the pipeline to the matrix
# normalized_matrix = pipeline.fit_transform(matrix)

# print(normalized_matrix)

## Идея:
- А почему бы перед использованием логистической регрессии не применить PCA к матрице текстов TF-IDF?
1. Классический PCA из sklearn не работает с sparse matrix,

2. Потому пользуемся альтернативой -  TruncatedSVD (концептуально все точно так же)

In [37]:
class PCA_CONSTS:
  def __init__(self):
    self.n_components=[100*i for i in range(1, 20+1)]

PCA_CONSTS_ = PCA_CONSTS()

In [38]:
PCA_CONSTS_.n_components

[100,
 200,
 300,
 400,
 500,
 600,
 700,
 800,
 900,
 1000,
 1100,
 1200,
 1300,
 1400,
 1500,
 1600,
 1700,
 1800,
 1900,
 2000]

In [32]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_2vec = TfidfVectorizer(
#     tokenizer=tokenizer.tokenize,
#     max_df=VECT_CONSTS.MAX_DF,
#     min_df=VECT_CONSTS.MIN_DF,
#     ngram_range=VECT_CONSTS.NGRAM_RANGE,
#     # preprocessor=lemmatizer_wiht_pos_tag
# )

# tfidf_2vec.fit_transform(train_source['data'])

In [33]:
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score, make_scorer


f1 = make_scorer(f1_score , average='weighted')

In [34]:
# # pipeline_basic = Pipeline([
# #     ('vect', TfidfVectorizer(
# #       tokenizer=tokenizer.tokenize,
# #     max_df=VECT_CONSTS.MAX_DF,
# #     min_df=VECT_CONSTS.MIN_DF,
# #     ngram_range=VECT_CONSTS.NGRAM_RANGE,
# #     # norm=None,
# #     # preprocessor=lemmatizer_wiht_pos_tag
# #     )),

# #     ('norm',
# #     #  StandardScaler(with_mean=False)
# #      ColumnNormalizer()
# #     ),

# #     ('cls', LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', max_iter=20))
# #     ])
# # sklearn_pipeline_basic.fit(train_source['data'], train_source['target']);

# pipeline_pca_gs = Pipeline([
#     ('vect', TfidfVectorizer(
#       tokenizer=tokenizer.tokenize,
#     max_df=VECT_CONSTS.MAX_DF,
#     min_df=VECT_CONSTS.MIN_DF,
#     ngram_range=VECT_CONSTS.NGRAM_RANGE,
#     # norm=None,
#     # preprocessor=lemmatizer_wiht_pos_tag
#     )),

#     # ('norm',
#     # #  StandardScaler(with_mean=False)
#     #  ColumnNormalizer()
#     # ),

#     # используем GridSearch для выбора оптимальной размерности TruncatedSVD (PCA для разреженных матриц)
#     ('cls_with_pca_gs',
#      GridSearchCV(Pipeline([
#       ('pca', TruncatedSVD()),
#       ('cls', LogisticRegression(max_iter=20))
#       ]),
#         param_grid={'pca__n_components': PCA_CONSTS_.n_components},
#         scoring=f1,
#         verbose=3,  # Set verbose to 1 for detailed logging
#         n_jobs=1  # Use multiple cores for faster grid search
#       ))
#     ])

# pipeline_pca_gs.fit(train_source['data'], train_source['target']);

In [54]:
i = 15

print('Число главных компонент:', PCA_CONSTS_.n_components[i], sep=' ')

Число главных компонент: 1600


In [ ]:
# best_cls = pipeline_pca_gs
best_cls = Pipeline([
    ('vect', TfidfVectorizer(
      tokenizer=tokenizer.tokenize,
    max_df=VECT_CONSTS.MAX_DF,
    min_df=VECT_CONSTS.MIN_DF,
    ngram_range=VECT_CONSTS.NGRAM_RANGE,
    # norm=None,
    # preprocessor=lemmatizer_wiht_pos_tag
    )),

    ('norm',
     StandardScaler(with_mean=False)
    #  ColumnNormalizer()
    ),

    ('pca', TruncatedSVD(n_components=PCA_CONSTS_.n_components[i], algorithm='arpack')),
    ('cls', LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', max_iter=20))
], verbose=True)

best_cls.fit(train_source['data'], train_source['target']);

[Pipeline] .............. (step 1 of 4) Processing vect, total=  13.7s
[Pipeline] .............. (step 2 of 4) Processing norm, total=   0.3s


In [ ]:
sklearn_train_pred = best_cls.predict_proba(train_source['data'])
sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(train_source['target']))
print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
print('Доля верных ответов', accuracy_score(train_source['target'], sklearn_train_pred.argmax(-1)))
print()

sklearn_test_pred = best_cls.predict_proba(test_source['data'])
sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(test_source['target']))
print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
print('Доля верных ответов', accuracy_score(test_source['target'], sklearn_test_pred.argmax(-1)))

увы, просто логрег после pca показывает себя очень плохо: для достижения результата в 80% необходимо 2000 главных компонент <br> :(